In [14]:
import spacy
import requests
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("HUGGINGFACE_API_KEY")

nlp = spacy.load("en_core_web_sm")
api_url = "https://api-inference.huggingface.co/models/google/flan-t5-small"





In [15]:
def pre(text):
    doc = nlp(text)
    return "".join(token.text+" " for token in doc )
    

print(pre("I am a student."))

I am a student . 


In [16]:
def payload(doc):
    headers = {
    "Authorization":f"Bearer {api_key}",
    }
    payload = {
        
        "inputs":doc,
        
    }

    response = requests.post(api_url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(f"Error: {response.text}")
        return None
    

In [17]:
def main():
    text = input("What happened macha?")
    ll = pre(text)
    print(payload(ll))

    

In [18]:
main()

Error: 404
Error: Not Found
None
